This experiment is done on the ground truth. The ground truth is created based on the NMVW constituents' display names and the rdfs:labels retrieved from the wikidata. It only contains constituesnts where the wikidata identifies is known.

In [25]:
import sys
sys.path.append("..") 

In [26]:
import pickle, pandas
from tqdm import tqdm

with open('data/ground_truth.pkl', 'rb') as file:
    data = pickle.load(file)

In [27]:
candidates = set([str(item) for _, row in data.iterrows() for item in row['wiki_label']])

In [28]:
len(data.index)

6178

In [29]:
len(list(set(candidates)))

11501

# Exact string matching

In [5]:
def exact_string_match(df, candidate_list):
    match = ["NO" for i in range(len(df.index))]
    match_results = []
    
    for i, row in df.iterrows():
        row_match_results = list()
        row_match_results = list(set([str(row['name_label'])]).intersection(set(candidates)))
        if len(row_match_results) > 0:
            match[i] = "YES"

        match_results.append(row_match_results)
        
    temp_df = pandas.DataFrame({'retrieved_names': match_results, 'match':match})
    result_table = pandas.concat([df, temp_df], axis=1)

    return result_table 

In [6]:
result = exact_string_match(data, candidates)

In [7]:
len(result.loc[result["match"] == "YES"])

3360

In [8]:
result.loc[result["match"] == "NO"]

,nmvw_uri,name_label,wiki_uri,wiki_label,retrieved_names,match
1,https://hdl.handle.net/20.500.11840/pi1020,Coenraad-Uhlig,http://www.wikidata.org/entity/Q86817941,"[Vera Coenraad-Uhlig, Vera Coenraad-Uhlig]",[],NO
2,https://hdl.handle.net/20.500.11840/pi1034,Dhr. W. Coolhaas,http://www.wikidata.org/entity/Q82570803,"[W. Coolhaas, W. Coolhaas]",[],NO
3,https://hdl.handle.net/20.500.11840/pi1036,Dhr. H. Cooymans,http://www.wikidata.org/entity/Q86735049,"[H. Cooymans, H. Cooymans]",[],NO
4,https://hdl.handle.net/20.500.11840/pi1049,Prof.dr. L.Ph. (Louis Philibert) le Cosquino d...,http://www.wikidata.org/entity/Q55022277,"[Louis Philibert le Cosquino de Bussy, Louis P...",[],NO
5,https://hdl.handle.net/20.500.11840/pi1054,Drs. F.M. (Frits) Cowan,http://www.wikidata.org/entity/Q82570805,"[Frits Cowan, Frits Cowan]",[],NO
...,...,...,...,...,...,...
6169,https://hdl.handle.net/20.500.11840/pi82779,A.M. (Adriaan Maarten) Montijn,http://www.wikidata.org/entity/Q20978414,"[Adriaan Maarten Montijn, Adriaan Maarten Mont...",[],NO
6171,https://hdl.handle.net/20.500.11840/pi82824,Ãmile Louis Duhousset,http://www.wikidata.org/wiki/Q21495507,[],[],NO
6173,https://hdl.handle.net/20.500.11840/pi82884,Dr. J.M. Geerts,https://www.wikidata.org/entity/Q114399444,[],[],NO
6176,https://hdl.handle.net/20.500.11840/pi900,Johan Theodorus Broekhuijse,http://www.wikidata.org/entity/Q2580232,"[Jan Broekhuijse, Jan Broekhuijse, Jan Broekhu...",[],NO


In [9]:
with open("results/NMVWtoWikiExactMatchResults.pkl", "wb") as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
df = pandas.read_pickle("results/NMVWtoWikiExactMatchResults.pkl")
print(len(df.index))

6178


In [11]:
# Write code for calculating score

In [24]:
from calculate_result import calculate_result
calculate_result("results/NMVWtoWikiExactMatchResults.pkl")

Total query: 6178 
Correct correspondence count: 3346
Recall: 0.5415992230495306


Total query: 6178
Total retrieved: 3360
Correct correspondence count: 3346 
Precision: 0.9958333333333333


F-measure: 0.7016145942545607



(6178, 3360, 3346, 0.5415992230495306, 0.9958333333333333, 0.7016145942545607)

# Surname Matching

In [34]:
def matchLastName(df, candidate_list):
    match = ["NO" for i in range(len(df.index))]
    match_results = []

    for i, row in df.iterrows():
        row_match_results = list()
        for name in candidate_list:
            if list(str(row['name_label']).split(" "))[-1] == list(str(name).split(" "))[-1]:
                row_match_results.append(str(name))
                match[i] = "YES"

        match_results.append(row_match_results)

    temp_df = pandas.DataFrame({'retrieved_names': match_results, 'match':match})
    result_table = pandas.concat([df, temp_df], axis=1)
    
    return result_table


In [35]:
result = matchLastName(data, candidates)

In [36]:
len(result.loc[result["match"] == "YES"])

5245

In [37]:
result.loc[result["match"] == "NO"]

,nmvw_uri,name_label,wiki_uri,wiki_label,retrieved_names,match
7,https://hdl.handle.net/20.500.11840/pi1095,H.T. (Henri Titus) DamstÃ©,http://www.wikidata.org/entity/Q75288654,"[Henri Titus Damsté, Henri Titus Damsté, Henri...",[],NO
21,https://hdl.handle.net/20.500.11840/pi1369,H.D. (Hieronijmus David) Fraenkel,http://www.wikidata.org/entity/Q60238568,"[Hieronymus D. Fraenkl, Hieronymus D. Fraenkl,...",[],NO
24,https://hdl.handle.net/20.500.11840/pi1439,Ir. D.G. Geyskes,http://www.wikidata.org/entity/Q2415208,"[Dirk Cornelis Geijskes, Dirk Cornelis Geijske...",[],NO
48,https://hdl.handle.net/20.500.11840/pi2024,Jvr. T. (Tyra) de Kleen,http://www.wikidata.org/entity/Q84498628,[],[],NO
57,https://hdl.handle.net/20.500.11840/pi20521,Abu Subhi Al- Tinawi,http://www.wikidata.org/entity/Q28877580,"[Abu Subhi Al-Tinawi, Abu Subhi Al-Tinawi, Abu...",[],NO
...,...,...,...,...,...,...
6154,https://hdl.handle.net/20.500.11840/pi82453,P. (Paul) Dittrich,https://commons.wikimedia.org/wiki/Category:P....,[],[],NO
6166,https://hdl.handle.net/20.500.11840/pi8274,Jan DaniÃ«l Beynon (1830-1877),http://www.wikidata.org/entity/Q26132769,"[Jan Daniel Beynon, Jan Daniel Beynon, Jan Dan...",[],NO
6167,https://hdl.handle.net/20.500.11840/pi82744,Wilhelm Ivens,http://www.wikidata.org/wiki/Q2097087,[],[],NO
6171,https://hdl.handle.net/20.500.11840/pi82824,Ãmile Louis Duhousset,http://www.wikidata.org/wiki/Q21495507,[],[],NO


In [38]:
with open("results/NMVWtoWikiSurnameMatchResults.pkl", "wb") as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
df = pandas.read_pickle("results/NMVWtoWikiSurnameMatchResults.pkl")
print(len(df.index))

6178


In [40]:
from calculate_result import calculate_result
calculate_result("results/NMVWtoWikiSurnameMatchResults.pkl")

Total query: 6178 
Correct correspondence count: 4991
Recall: 0.8078666235027517


Total query: 6178
Total retrieved: 9613
Correct correspondence count: 5833 
Precision: 0.6067824820555497


F-measure: 0.6930330822714871



(6178, 9613, 5833, 0.8078666235027517, 0.6067824820555497, 0.6930330822714871)

# Abbreviation Matching

In [48]:
def match_with_abbreviation(df, candidate_list):
    def get_initials(name):
        if len(name.split(" ")) < 1:
            print(name)
            return

        names = name.split(' ')
        try:
            return ' '.join([f"{n[0]}." for n in names])
        except Exception as e:
            print(f'{name} has {str(e)} error')

    abbreviation_list = list()
    for i, row in df.iterrows():
        firstnames = get_initials(" ".join(str(row['name_label']).split(" ")[:-1]))
        if firstnames:
            initial_and_surname = firstnames + " " + str(row['name_label']).split(" ")[-1]
        else:
            initial_and_surname = str(row['name_label']).split(" ")[-1]
        # print(f"{str(row['name_label'])} --> {initial_and_surname}")
        abbreviation_list.append(initial_and_surname)

    temp_df = pandas.DataFrame({'abbreviations': abbreviation_list})
    df = pandas.concat([df, temp_df], axis=1)
    
    abbrv_candidate_list = list()
    for item in candidate_list:
        firstnames = get_initials(" ".join(str(item).split(" ")[:-1]))
        if firstnames:
            initial_and_surname = firstnames + " " + str(row).split(" ")[-1]
        else:
            initial_and_surname = str(row).split(" ")[-1]
        abbrv_candidate_list.append(initial_and_surname)
    
    match = ["NO" for i in range(len(df.index))]
    match_results = []
    # matching
    for i, row in df.iterrows():
        row_match_results = list()
        row_match_results = list(set([str(row['abbreviations'])]).intersection(set(abbrv_candidate_list)))
        if len(row_match_results) > 0:
            match[i] = "YES"

        match_results.append(row_match_results)
        
    temp_df = pandas.DataFrame({'retrieved_names': match_results, 'match':match})
    result_table = pandas.concat([df, temp_df], axis=1)

    return result_table

In [51]:
result = match_with_abbreviation(data, candidates)

 has string index out of range error
Dhr. J.E. (Johan Ernst)  has string index out of range error
 has string index out of range error
 has string index out of range error
Mr.  J.W. has string index out of range error
 has string index out of range error
J.I.  (Jowa Imre) has string index out of range error
G.A.J. (Gijsbertus Adrian Johan)  van der has string index out of range error
Mevr. S.C. (Susanna Carolina) De Wildt  - de has string index out of range error
 has string index out of range error
Dewa  PT. has string index out of range error
Mehmet  has string index out of range error
 has string index out of range error
 has string index out of range error
Ibu  has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range 

 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
T. (Turgut) Reshadi  has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index 

 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 

 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 

 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 

 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 

 has string index out of range error
 has string index out of range error
Jan  Sterk has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
Directie Nw-Guinea v Ministerie v  Binnenl has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
Johannes  Baer has string index out of range error
 has string index out of range error
 has string index out of range error
 has stri

 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 

 has string index out of range error
 has string index out of range error
Julius  Muller has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
Willem Bernardus  IJzerdraat has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error
 has string index out of range error


In [52]:
result

,nmvw_uri,name_label,wiki_uri,wiki_label,abbreviations,retrieved_names,match
0,https://hdl.handle.net/20.500.11840/pi1008,Gabriel Clark-Brown,http://www.wikidata.org/entity/Q82570739,"[Gabriel Clark-Brown, Gabriel Clark-Brown]",G. Clark-Brown,[],NO
1,https://hdl.handle.net/20.500.11840/pi1020,Coenraad-Uhlig,http://www.wikidata.org/entity/Q86817941,"[Vera Coenraad-Uhlig, Vera Coenraad-Uhlig]",Coenraad-Uhlig,[],NO
2,https://hdl.handle.net/20.500.11840/pi1034,Dhr. W. Coolhaas,http://www.wikidata.org/entity/Q82570803,"[W. Coolhaas, W. Coolhaas]",D. W. Coolhaas,[],NO
3,https://hdl.handle.net/20.500.11840/pi1036,Dhr. H. Cooymans,http://www.wikidata.org/entity/Q86735049,"[H. Cooymans, H. Cooymans]",D. H. Cooymans,[],NO
4,https://hdl.handle.net/20.500.11840/pi1049,Prof.dr. L.Ph. (Louis Philibert) le Cosquino d...,http://www.wikidata.org/entity/Q55022277,"[Louis Philibert le Cosquino de Bussy, Louis P...",P. L. (. P. l. C. d. Bussy,[],NO
...,...,...,...,...,...,...,...
6173,https://hdl.handle.net/20.500.11840/pi82884,Dr. J.M. Geerts,https://www.wikidata.org/entity/Q114399444,[],D. J. Geerts,[],NO
6174,https://hdl.handle.net/20.500.11840/pi838,Charles Ralph Boxer,http://www.wikidata.org/entity/Q954203,"[تشارلز رالف بوكسير, Charles Ralph Boxer, C. R...",C. R. Boxer,[],NO
6175,https://hdl.handle.net/20.500.11840/pi875,Dolf Breetvelt,http://www.wikidata.org/entity/Q13573764,"[Dolf Breetvelt, Dolf Breetvelt, Dolf Breetvel...",D. Breetvelt,[],NO
6176,https://hdl.handle.net/20.500.11840/pi900,Johan Theodorus Broekhuijse,http://www.wikidata.org/entity/Q2580232,"[Jan Broekhuijse, Jan Broekhuijse, Jan Broekhu...",J. T. Broekhuijse,[],NO


In [53]:
with open("results/NMVWtoWikiSurnameMatchResults.pkl", "wb") as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [54]:
df = pandas.read_pickle("results/NMVWtoWikiSurnameMatchResults.pkl")
print(len(df.index))

6178


In [55]:
from calculate_result import calculate_result
calculate_result("results/NMVWtoWikiSurnameMatchResults.pkl")

Total query: 6178 
Correct correspondence count: 0
Recall: 0.0


The total retrieved is: 0!!!!
Total query: 6178
Total retrieved: 0
Correct correspondence count: 0 


UnboundLocalError: local variable 'p' referenced before assignment